# Continuous Time SIR SDE using `DifferentialEquations`

### Packages

In [ ]:
# Data Manipulation
using DataFrames

# Statistics
using Distributions
using Random

# Data Visualization
using StatPlots
using PlotThemes

# Numerical Computation
using DifferentialEquations
using SimpleDiffEq

┌ Info: Precompiling StatPlots [60ddc479-9b66-56df-82fc-76a74619b69c]
└ @ Base loading.jl:1260


### Flow

In [ ]:
function Φ!(du,u,p,t)
    (S,I,R) = u
    (β,c,γ,δt) = p
    N = S+I+R
    ifrac = β*c*I/N*S*δt
    rfrac = γ*I*δt
    ifrac_noise = sqrt(ifrac)*rand(Normal(0,1))
    rfrac_noise = sqrt(rfrac)*rand(Normal(0,1))
    @inbounds begin
        du[1] = S-(ifrac+ifrac_noise)
        du[2] = I+(ifrac+ifrac_noise) - (rfrac + rfrac_noise)
        du[3] = R+(rfrac+rfrac_noise)
    end
    for i in 1:3
        if du[i] < 0 du[i]=0 end
    end
    nothing
end

### Parameters

In [ ]:
# From discrete to continuous time
δt = 0.1
nsteps = 400
tmax = nsteps*δt
tspan = (0.0,nsteps)
t = 0.0:δt:tmax;

𝒯 = tspan;               # Time
ℬ = [990.0,10.0,0.0]     # Initial condition
𝒫 = [0.05,10.0,0.25,δt]; # Model parameters

### Solution

In [ ]:
Random.seed!(1234); # Set a random seed

# Problem Definition
problem = DiscreteProblem(Φ!, ℬ, 𝒯, 𝒫);

# Problem Solution
solution = solve(problem, solver=FunctionMap);

### Visualization 

In [ ]:
# Convert the output to data frame
data = DataFrame(solution')
data[!,:t] = t;

# Select Theme
theme(:default)

@df data plot(:t,
    [:x1 :x2 :x3],
    title = "SIR SDE",
    label=["S" "I" "R"],
    legend=:top,
    xlabel="Time",
    ylabel="Number")